In [1]:
import cv2
import math
import pandas as pd
from moviepy import VideoFileClip
import os

In [2]:
import sys
from moviepy import VideoFileClip

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()
else:
    print('aucun module')

aucun module


In [43]:
# Define project information
# PROJECT_ID = "lunar-airport-418323"  # @param {type:"string"}
# LOCATION = "us-central1"  # @param {type:"string"}
PROJECT_ID = "pixar_bucket_test"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [12]:
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Image,
    Part,
)

In [13]:
# Define GenerationConfig with desired parameters
generation_config = GenerationConfig(
    temperature=0.0,  # Adjust temperature between 0.0 (deterministic) and 1.0 (more random)
    #top_p=0.3,  # Adjust top_p between 0.0 (likely tokens) and 1.0 (considers all tokens)
    #top_k= 32
)

In [ ]:
multimodal_model = GenerativeModel("gemini-1.0-pro-vision")

In [15]:
captions_list = []
sample_list = []

In [23]:
import os

# Remplace par le chemin absolu vers ta clé
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"E:\MiB\MindBridge\data\natural-scenes-dataset\webdataset_avg_split\train\tokyo_griffin_463804_h6_aa546859ce5a.json"


In [19]:
from google.cloud import aiplatform
aiplatform.init()
print("Authentification réussie !")


Authentification réussie !


In [35]:
from google.cloud import storage
client = storage.Client()
bucket = client.bucket("pixar_bucket_test")
blob = bucket.blob("sample0000000013 - Copie.jpg")

if blob.exists():
    print("Le fichier existe !")
else:
    print("Fichier introuvable.")


Forbidden: 403 GET https://storage.googleapis.com/storage/v1/b/pixar_bucket_test/o/sample0000000013%20-%20Copie.jpg?fields=name&prettyPrint=false: vertex-access@tokyo-griffin-463804-h6.iam.gserviceaccount.com does not have storage.objects.get access to the Google Cloud Storage object. Permission 'storage.objects.get' denied on resource (or it may not exist).

In [ ]:
import time

prompt = """
Describe this image with less than 77 characters including spaces.
"""

for i in range(0, 48):
    frame_nb = i
    sample = f"sample00000000{frame_nb}"
    frame_uri = f"gs://pixar_bucket_test/sample00000000{frame_nb} - Copie.jpg"
    frame = Part.from_uri(frame_uri, mime_type="image/jpeg")
    contents = [frame, prompt]
    try:
        responses = multimodal_model.generate_content(contents, generation_config=generation_config, stream=True)
        # On convertit le générateur en liste pour pouvoir l'utiliser plusieurs fois
        responses_list = list(responses)
        for response in responses_list:
            if hasattr(response, "text"):
                print(response.text)
        caption = [response.text for response in responses_list if hasattr(response, "text")]
    except Exception as e:
        print(f"Erreur pour {sample}: {e}")
        caption = ["ERROR"]
    captions_list.append(caption)
    sample_list.append(sample)
    time.sleep(1)

Erreur pour sample00000000526: 'list' object has no attribute 'get'
Erreur pour sample00000000527: 'list' object has no attribute 'get'
Erreur pour sample00000000528: 'list' object has no attribute 'get'
Erreur pour sample00000000529: 'list' object has no attribute 'get'
Erreur pour sample00000000530: 'list' object has no attribute 'get'
Erreur pour sample00000000531: 'list' object has no attribute 'get'
Erreur pour sample00000000532: 'list' object has no attribute 'get'
Erreur pour sample00000000533: 'list' object has no attribute 'get'
Erreur pour sample00000000534: 'list' object has no attribute 'get'
Erreur pour sample00000000535: 'list' object has no attribute 'get'
Erreur pour sample00000000536: 'list' object has no attribute 'get'
Erreur pour sample00000000537: 'list' object has no attribute 'get'
Erreur pour sample00000000538: 'list' object has no attribute 'get'
Erreur pour sample00000000539: 'list' object has no attribute 'get'
Erreur pour sample00000000540: 'list' object has

KeyboardInterrupt: 

In [51]:
import time

prompt = """
Describe this image with less than 77 characters including spaces.
"""

#Generate a caption for every images in the bucket
for i in range(526,828):
    frame_nb = i
    sample = f"sample00000000{frame_nb}"
    frame_uri = f"gs://pixar_bucket_test/sample00000000{frame_nb} - Copie.jpg"
    frame = Part.from_uri(frame_uri, mime_type="image/jpeg")
    contents = [frame, prompt]
    print(contents)
    responses = multimodal_model.generate_content(contents, generation_config=generation_config, stream=True)
    for response in responses:
        print(response.text)
    caption =[response.text for response in responses]
    captions_list.append(caption)
    sample_list.append(sample)

    # Introduce a delay to avoid exceeding the quota
    time.sleep(1)  # Adjust the sleep duration as necessary

[file_data {
  mime_type: "image/jpeg"
  file_uri: "gs://pixar_bucket_test/sample00000000526 - Copie.jpg"
}
, '\nDescribe this image with less than 77 characters including spaces.\n']


AttributeError: 'list' object has no attribute 'get'

In [36]:
#test for 1 image:

prompt = """
Describe this image with less than 77 characters including spaces.
"""
frame_nb = 526
frame_uri = f"gs://frames_vid/sample00000000{frame_nb}.jpg" 
frame = Part.from_uri(frame_uri, mime_type="image/jpeg")
contents = [frame, prompt]
responses = multimodal_model.generate_content(contents, generation_config=generation_config, stream=True)
caption =[response.text for response in responses]
print(caption)

AttributeError: 'list' object has no attribute 'get'

In [34]:
print("Len caption_list:", len(captions_list), "Len sample_list:", len(sample_list), "Dernier element sample_list:", sample_list[-1])

Len caption_list: 827 Len sample_list: 827 Dernier element sample_list: sample00000000827


In [35]:
captions_list[-1]

[' A road cut through a rocky hill.', '']

In [36]:
import pandas as pd
captions_dict = {'sampleID' : sample_list, 'caption' : captions_list}
captions_df = pd.DataFrame(captions_dict)
captions_df

,sampleID,caption
0,sample000000000,[ A car is driving on a road next to the ocean...
1,sample000000001,[ A car is driving on a road next to the ocean...
2,sample000000002,[ A car is driving on a road next to the ocean...
3,sample000000003,[ A car is driving on a road next to the ocean...
4,sample000000004,[ A car is driving on a road next to the ocean...
...,...,...
822,sample00000000823,"[ A road cut through a rocky hill., ]"
823,sample00000000824,[ A road cut through a rock formation with a b...
824,sample00000000825,"[ A road cut through a rock formation., ]"
825,sample00000000826,"[ A road cut through a rocky hill., ]"


In [37]:
captions_df.to_csv('C:/Users/hennecol/Documents/FMRI experiment/FINAL FOLDER/Lucas/MindBridge/data/natural-scenes-dataset/nsddata_stimuli/stimuli/nsd/annotations/captions77char.csv')

In [3]:
captions_df = pd.read_csv(r"E:\MiB\MindBridge\data\natural-scenes-dataset\nsddata_stimuli\stimuli\nsd\annotations\captions77char.csv")

In [5]:
import pandas as pd

# Supposons que la colonne s'appelle 'sampleID'
captions_df['sampleID'] = captions_df['sampleID'].str.replace('.jpg', '', regex=False)

captions_df

,sampleID,caption
0,sample000000000,"A blank, dark grey background with no visible ..."
1,sample000000001,Pixar Animation Studios logo with a blue gradi...
2,sample000000002,Surreal sky scene with clouds and flying objec...
3,sample000000003,Animated character riding a giant creature in ...
4,sample000000004,Blurry image with person reaching out to cloud...
5,sample000000005,A small brown puppy peeks through fluffy pink ...
6,sample000000006,"Pink cartoon pig with crown, sitting on cloud ..."
7,sample000000007,A stork carries a baby wrapped in a bundle thr...
8,sample000000008,A whimsical scene with figures riding fluffy c...
9,sample000000009,"A stone statue of a cow overlooks a cloudy, my..."


In [6]:
captions_df.to_csv(r"E:\MiB\MindBridge\data\natural-scenes-dataset\nsddata_stimuli\stimuli\nsd\annotations\captions77char.csv")

In [7]:
#List of the samples without caption, gemini unable to generate a caption for these images

import pandas as pd

samples = captions_df['sampleID']

# Extract sample numbers
sample_numbers = samples.str.extract('(\d+)$').astype(int)[0]

# Create a reference list of all expected numbers
expected_numbers = set(range(sample_numbers.min(), sample_numbers.max() + 1))

# Identify missing numbers
missing_numbers = expected_numbers - set(sample_numbers)

# Show missing numbers
print(sorted(missing_numbers))


[]


<>:8: SyntaxWarning: invalid escape sequence '\d'
<>:8: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Rémi\AppData\Local\Temp\ipykernel_18204\3228297796.py:8: SyntaxWarning: invalid escape sequence '\d'
  sample_numbers = samples.str.extract('(\d+)$').astype(int)[0]


In [15]:
# Function to transform data and put it into MindBridge input format
def transform_dataframe(df):
    transformed_data = []
    for index, row in df.iterrows():
        sample_number = int(row['sampleID'].split('sample')[-1])
        caption = row['caption']
        # caption = " ".join(row['caption']).strip()  # Concatenate and remove leading and trailing spaces
        # caption = " ".join(caption.split())  # Remove extra spaces between words
        new_entry = {
            'image_id': 1000000 + sample_number,
            'id': 1000000 + sample_number,
            'caption': caption
        }
        transformed_data.append(new_entry)
    return transformed_data

In [16]:
result = transform_dataframe(captions_df)

In [17]:
result[40]

{'image_id': 1000040,
 'id': 1000040,
 'caption': 'A warrior stands on a rocky hill under a dramatic, cloudy sky.'}

In [18]:
import json
json_file_path = r"E:\MiB\MindBridge\data\natural-scenes-dataset\nsddata_stimuli\stimuli\nsd\annotations\mycaptions77char.json"

# Save the result as JSON
with open(json_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(result, json_file, ensure_ascii=False, indent=4)

# Confirmation of registration
print(f"Le fichier JSON a été enregistré à {json_file_path}")

Le fichier JSON a été enregistré à E:\MiB\MindBridge\data\natural-scenes-dataset\nsddata_stimuli\stimuli\nsd\annotations\mycaptions77char.json


In [21]:
#Complete the json file with our annotations

import json

# JSON file paths
generated_json_path = r"E:\MiB\MindBridge\data\natural-scenes-dataset\nsddata_stimuli\stimuli\nsd\annotations\mycaptions77char.json"
annotations_json_path = r"E:\MiB\MindBridge\data\natural-scenes-dataset\nsddata_stimuli\stimuli\nsd\annotations\captions_val2017.json" #Change val and then train

# Load the previously generated JSON file
with open(generated_json_path, 'r', encoding='utf-8') as json_file:
    generated_data = json.load(json_file)

# Load the existing annotations JSON file
with open(annotations_json_path, 'r', encoding='utf-8') as json_file:
    annotations_data = json.load(json_file)

# Add the new annotations to the annotations list
annotations_data['annotations'].extend(generated_data)

# Save updated JSON file
output_file_path = r'E:\MiB\MindBridge\data\natural-scenes-dataset\nsddata_stimuli\stimuli\nsd\annotations\updated_captions77char_val2017.json'
with open(output_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(annotations_data, json_file, ensure_ascii=False, indent=4)

# Confirmation of registration
print(f"Le fichier JSON mis à jour a été enregistré à {output_file_path}")

Le fichier JSON mis à jour a été enregistré à E:\MiB\MindBridge\data\natural-scenes-dataset\nsddata_stimuli\stimuli\nsd\annotations\updated_captions77char_val2017.json


In [22]:
import json

# Path to the train annotations file
train_annotations_path =r'E:\MiB\MindBridge\data\natural-scenes-dataset\nsddata_stimuli\stimuli\nsd\annotations\updated_captions77char_val2017.json'

# Check if the file has the same structure
try:
    with open(train_annotations_path, 'r', encoding='utf-8') as json_file:
        train_annotations_data = json.load(json_file)
        print(train_annotations_data['annotations'][-1])
except Exception as e:
    print(f"An error occurred: {e}")


{'image_id': 1000048, 'id': 1000048, 'caption': 'A blank, black image with no discernible content or details.'}
